We scanned *a lot* of teeth for the ZMK guys and gals.
Let's programmatically generate overview images of them.

In [1]:
import platform
import os
import glob
import pandas
# import imageio
import numpy
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
# import seaborn
import dask
import dask_image.imread
from dask.distributed import Client
client = Client()
# Then go to http://localhost:8787/status
# from numcodecs import Blosc
from tqdm import notebook
# import skimage.filters
# import skimage.morphology
# import skimage.segmentation
# import scipy.ndimage

In [2]:
print('You can seee what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

You can seee what DASK is doing at "http://localhost:8787/status"


In [3]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit

In [4]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [5]:
# Different locations if running either on Linux or Windows
FastSSD = False
# to speed things up significantly
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
    else:
        BasePath = os.path.join(os.sep, 'home', 'habi', '1272')
else:
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        if 'anaklin' in platform.node():
            BasePath = os.path.join('S:\\')
        else:
            BasePath = os.path.join('D:\\Results')
Root = os.path.join(BasePath, 'ZMK')
print('We are loading all the data from %s' % Root)

We are loading all the data from S:\ZMK


In [6]:
if 'Linux' in platform.system():
    tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\')
    else:
        tmp = os.path.join('D:\\')
dask.config.set({'temporary_directory': os.path.join(tmp, 'tmp')})

In [7]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [8]:
# Look only for folders: https://stackoverflow.com/a/38216530
Data['Folder'] = glob.glob(os.path.join(Root, 'ToothBattallion', '*' + os.path.sep))

In [9]:
print('We found %s tooth folders in %s' % (len(Data), Root))

We found 104 tooth folders in S:\ZMK


In [10]:
# Look for the relevant logfiles
Data['LogFile'] = [sorted(glob.glob(os.path.join(f, '*.log')))[0] for f in Data['Folder']]

In [11]:
Data['Sample'] = [os.path.splitext(os.path.basename(l))[0] for l in Data['LogFile']]

In [12]:
# Proper sorting *with* leading zeros :)
Data.sort_values(by=['Sample'], inplace=True)

In [13]:
# Reconstruction filenames
Data['OutputNameRec'] = [os.path.join(f, sample + '_rec.zarr') for f, sample in zip(Data.Folder, Data.Sample)]

In [14]:
# Load the reconstructions a zarr arrays
Reconstructions = [dask.array.from_zarr(file) for file in Data['OutputNameRec']]

In [15]:
# How big are the datasets?
Data['Size'] = [rec.shape for rec in Reconstructions]

In [16]:
# The three cardinal directions
directions = ['Axial',
              'Coronal',
              'Sagittal']

In [17]:
# Cropped filenames
Data['OutputNameRecCrop'] = [os.path.join(f, sample + '_rec_crop.zarr') for f, sample in zip(Data.Folder, Data.Sample)]

In [18]:
# Load the cropped zarr arrays 
CroppedReconstructions = [dask.array.from_zarr(file) for file in Data['OutputNameRecCrop']]

In [19]:
# Update the size
Data['Size'] = [rec.shape for rec in CroppedReconstructions]

In [20]:
# Pulpa file names
Data['OutputNamePulpa'] = [os.path.join(f, sample + '_pulpa.zarr') for f, sample in zip(Data.Folder, Data.Sample)]

In [21]:
# Load the cropped zarr arrays back in as *reconstructions*
Pulpas = [dask.array.from_zarr(file) for file in Data['OutputNamePulpa']]

In [22]:
# 3D-Display

In [23]:
import itkwidgets
from itkwidgets import view  # 3d viewer

In [24]:
whichone=44
Data.Sample[whichone]

'Tooth045'

In [25]:
TeethPulpa = numpy.maximum(Pulpas[whichone][1:] * 255, CroppedReconstructions[whichone][1:]).astype('uint8')

In [26]:
view(TeethPulpa,
     annotations=False,
     ui_collapsed=True,
     rotate=False,
     gradient_opacity=0.618)

Viewer(annotations=False, geometries=[], gradient_opacity=0.618, point_sets=[], rendered_image=<itk.itkImagePy…

In [27]:
# viewer = view(Reconstructions[whichone][::25,::25,::25],
#               annotations=False,
#               ui_collapsed=True,
#               rotate=False,
#               gradient_opacity=0.618)
# # Rotate camera, based on https://git.io/JfjwT and *highly* empirical fiddling
# viewer.camera = numpy.array([[-700, 0,  90],
#                              [ 90,  0,  90],
#                              [ 0,  1,  0]], dtype=numpy.float32)
# viewer

In [28]:
# viewer.camera

In [29]:
# import ipywebrtc as webrtc
# import time

In [30]:
# viewer = view(TeethPulpa[::15,::15,::15].astype('uint8'),
#               annotations=False,
#               ui_collapsed=True,
#               rotate=False,
#               gradient_opacity=0.618)
# # Rotate camera, based on https://git.io/JfjwT
# viewer.camera = numpy.array([viewer.camera[0],
#                              viewer.camera[1],
#                              [-8.8914531e-01,  4.5637247e-01, -3.3833973e-02]], dtype=numpy.float32)
# viewer

In [31]:
# # https://github.com/InsightSoftwareConsortium/itkwidgets/issues/344#issuecomment-652735527
# from IPython.display import display

In [32]:
# for c, smpl in Data.iterrows():
#     print(smpl.Sample)
#     viewer = view(CroppedReconstructions[c][::25,::25,::25],
#                   annotations=False,
#                   ui_collapsed=True,
#                   rotate=False,
#                   gradient_opacity=0.618)
#     viewer.camera = numpy.array([[-700, 0,  90],
#                                  [ 90,  0,  90],
#                                  [ 0,  1,  0]], dtype=numpy.float32)    
#     display(viewer)
#     time.sleep(1)    
#     recorder = webrtc.ImageRecorder(stream=viewer,
#                                     filename=os.path.join(Data['Folder'][whichone],
#                                                           Data['Sample'][whichone] + '.3D.Reconstruction.png'),
#                                     autosave=True)
#     recorder
#     recorder.recording = True
#     time.sleep(1)
#     recorder.recording = False    

In [33]:
# display(viewer)

In [34]:
# for whichone, row in Data.iterrows():
#     print(row.Sample)
#     viewer.image = CroppedReconstructions[whichone][::25,::25,::25].compute()
#     recorder = webrtc.ImageRecorder(stream=viewer,
#                                     filename=os.path.join(Data['Folder'][whichone],
#                                                           Data['Sample'][whichone] + '.3D.Reconstruction.png'),
#                                     autosave=True)
#     recorder
#     recorder.recording = True
#     recorder.recording = False

In [35]:
# for c,smpl in Data.iterrows():
#     print(smpl.Sample)
#     viewer = view(CroppedReconstructions[c][::15,::15,::15],
#                   annotations=False,
#                   ui_collapsed=True,
#                   rotate=False,
#                   gradient_opacity=0.618)
#     display(viewer)
#     recorder = webrtc.ImageRecorder(stream=viewer,
#                                     filename=os.path.join(Data['Folder'][c],
#                                                           Data['Sample'][c] + '3D.Tooth.png'),
#                                     autosave=True)
#     recorder
#     recorder.recording = True
#     time.sleep(0.5)
#     recorder.recording = False

In [36]:
# # From Kevin: https://twitter.com/maartenbreddels/status/1042005230437756928
# import ipyvolume as ipv
# import numpy as np
# from matplotlib.pyplot import cm

In [37]:
# # download the dataset
# !wget -q https://www.dropbox.com/s/eqht79b7j4jqit2/petct.npz?dl=1 -O petct.npz

In [38]:
# full_scan = {k: v.swapaxes(0, 1)[::-1] for k,v in np.load('petct.npz').items()}
# print(list(full_scan.keys()))

In [39]:
# table_ct = cm.gray_r(np.linspace(0, 1, 255))
# table_ct[:50, 3] = 0 # make the lower values transparent
# table_ct[50:, 3] = np.linspace(0, 0.05, table_ct[50:].shape[0])
# tf_ct = ipv.TransferFunction(rgba=table_ct)

In [40]:
# ct_vol = ipv.quickvolshow(full_scan['ct_data'], 
#                           tf=tf_ct, lighting=False, 
#                           data_min=-1000, data_max=1000)
# ct_vol

In [41]:
# import ipywidgets

In [42]:
# import ipyvolume.pylab as p3
# import ipyvolume as ipv
# import IPython.display
# import ipyvolume
# import numpy as np
# import os

In [43]:
# ball = ipyvolume.examples.ball(shape=32, draw=False)

In [44]:
# fig = p3.figure(screen_capture_enabled=True)
# vol = p3.volshow(test)
# p3.show()

In [45]:
# import k3d

In [46]:
# tooth = k3d.volume(
#     Reconstructions[1],
# #     alpha_coef=1000,
# #     shadow='dynamic',
# #     samples=600,
# #     shadow_res=128,
# #     shadow_delay=50,
#     color_range=[50,2**8],
# #     color_map=(np.array(k3d.colormaps.matplotlib_color_maps.Gist_heat).reshape(-1,4)
# #                * np.array([1,1.75,1.75,1.75])).astype(np.float32),
# #     compression_level=9
# )
# plot = k3d.plot()
# plot += tooth
# plot.display()

In [47]:
# with open('tooth001.html','w') as fp:
#     fp.write(plot.snapshot)